In [17]:
import os
import re
import torch
import torch.nn as nn
import numpy
import torch.nn.functional as F
import pandas as pd
import matplotlib.pyplot as plt


In [18]:
def load_data(path="../train.tsv", length=500):
    df = pd.read_csv(path, sep="\t")[:length]
    df.drop(['PhraseId','SentenceId'], axis=1, inplace=True)
    x, y = df["Phrase"].values, df["Sentiment"].values
    return (x, y)

x, y = load_data("../train.tsv", 500)

def clean_text(x):
  # Removes special symbols and just keep
  # words in lower or upper form
  
  x = [i.lower() for i in x]
  x = [re.sub(r'[^A-Za-z]+', ' ', i) for i in x]
  
  return x

x = clean_text(x)
print(x[:5], y[:5])


['a series of escapades demonstrating the adage that what is good for the goose is also good for the gander some of which occasionally amuses but none of which amounts to much of a story ', 'a series of escapades demonstrating the adage that what is good for the goose', 'a series', 'a', 'series'] [1 2 2 2 2]


In [ ]:
import torchtext
from torchtext.legacy.data import Field
from torchtext.legacy.data import TabularDataset
from torchtext.vocab import GloVe
from torchtext.legacy.data import Iterator, BucketIterator
import torchtext.datasets
from torchtext.datasets import IMDB


TEXT = data.Field(tokenize=data.get_tokenizer('spacy'), 
                  init_token='<SOS>', eos_token='<EOS>',lower=True)

train, val, test = data.TabularDataset.splits(
        path='../', train='train.tsv', test='test.tsv', format='tsv',
        fields=[('Text', TEXT), ('Label', LABEL)])

TEXT.build_vocab(train)
#TEXT.build_vocab(train, vectors="glove.6B.100d")

In [19]:
def build_dict(x):
    ret = []
    for i in x:
        ret += i.split()
    
    return list(set(ret))

d = build_dict(x)

word2inx = {d[i]:i for i in range(len(d))}

length_dict = len(word2inx)

print(word2inx)


{'work': 0, 'for': 1, 'leave': 2, 'companion': 3, 'seeking': 4, 'you': 5, 'wong': 6, 'by': 7, 'should': 8, 'an': 9, 'betrayal': 10, 'have': 11, 'snow': 12, 'intrigue': 13, 'its': 14, 'our': 15, 'who': 16, 'arts': 17, 'chance': 18, 'dialogue': 19, 'fresnadillo': 20, 'performance': 21, 'ismail': 22, 'worth': 23, 'character': 24, 'childhood': 25, 'or': 26, 'love': 27, 'being': 28, 'comedy': 29, 'kong': 30, 'mess': 31, 'oedekerk': 32, 'path': 33, 'does': 34, 'midlife': 35, 'opera': 36, 'epic': 37, 'his': 38, 'joy': 39, 'modest': 40, 'trouble': 41, 'aggressive': 42, 'proves': 43, 'charmer': 44, 'sincere': 45, 'escapades': 46, 'whitewash': 47, 'positively': 48, 'earnest': 49, 'perspective': 50, 'murder': 51, 'thick': 52, 'escapism': 53, 'could': 54, 'us': 55, 'sense': 56, 'dogs': 57, 'be': 58, 'undergoing': 59, 'youth': 60, 'narratively': 61, 'recommend': 62, 'quotations': 63, 'like': 64, 'mr': 65, 's': 66, 'too': 67, 'importance': 68, 'sitting': 69, 'also': 70, 'high': 71, 'considers': 72, 

In [22]:
def sentence2vector(sentence):
    v = []
    for word in sentence.split():
        v.append(word2inx[word])
    return torch.Tensor(v)

test_input = sentence2vector(x[0])
test_target = y[0]

print(test_input, test_target)

def get_batch(x, batch_size=2, point=0):
    return  [sentence2vector(i) for i in x[point:point+batch_size]]

test_batch = get_batch(x)

print(test_batch)


tensor([109., 102., 149.,  46., 170., 138., 168., 199., 150., 167.,  86.,   1.,
        138., 205., 167.,  70.,  86.,   1., 138., 204., 182., 149., 183., 136.,
        196., 197.,  98., 149., 183.,  89., 190., 173., 149., 109., 135.]) 1
[tensor([109., 102., 149.,  46., 170., 138., 168., 199., 150., 167.,  86.,   1.,
        138., 205., 167.,  70.,  86.,   1., 138., 204., 182., 149., 183., 136.,
        196., 197.,  98., 149., 183.,  89., 190., 173., 149., 109., 135.]), tensor([109., 102., 149.,  46., 170., 138., 168., 199., 150., 167.,  86.,   1.,
        138., 205.])]


In [ ]:
class CNN(nn.Module):
    def __init__(self, input_size=length_dict, embed_dim=100, kernel_wins=[3,4,5], num_class):
        super(CNN, self).__init__()
        self.embedding = nn.Embedding(input_size, embed_dim, max_norm=True)
        self.convs = nn.ModuleList([nn.Conv2d(1, embed_dim, ) for size in kernel_wins])
        self.dropout = nn.Dropout(0.6)
        self.fc = nn.Linear(len(kernel_wins)*dim_channel, num_class)

    def forward(self, x):
        embeded_x = self.embedding(x)
        embeded_x.unsqueeze(1)
        con_x = [conv(emb_x) for conv in self.convs]
        pool_x = [F.max_pool1d(x.squeeze(-1), x.size()[2]) for x in con_x]
        
        fc_x = torch.cat(pool_x, dim=1)
        
        fc_x = fc_x.squeeze(-1)

        fc_x = self.dropout(fc_x)
        logit = self.fc(fc_x)
        return logit

def test_model():
    pass

test_model()


In [ ]:
def train():
    pass



In [ ]:
def test():
    pass